In [19]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re
from tqdm import tqdm
import ssl
import sys
import warnings
warnings.filterwarnings('ignore')

In [20]:
# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search,page,maxpage_t, sort, s_date, e_date, s_from, e_to):
    urls = []
    if page == maxpage_t:       # 최대페이지 1일 경우
        url = "https://search.naver.com/search.naver?where=news&query=" + search + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        urls.append(url)
    else:
        while page <= maxpage_t :
            url = "https://search.naver.com/search.naver?where=news&query=" + search + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
            urls.append(url)
            page += 10
    return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(url, headers=headers, verify=False)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

In [21]:
#####뉴스크롤링 시작#####

# 최대 페이지 수
maxpage = int(input("최대 크롤링할 페이지 수 입력하시오: "))
#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
# 검색 방식
sort = input("뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): ")    #관련도순=0  최신순=1  오래된순=2
# 시작날짜
s_date = input("시작날짜 입력(2019.01.04):")  #2019.01.04
# 끝나는 날짜
e_date = input("끝날짜 입력(2019.01.05):")   #2019.01.05

# 날짜 바꾸기
s_from = s_date.replace(".","")
e_to = e_date.replace(".","")

page = 1  
maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

# naver url 생성
urls = makeUrl(search,page,maxpage_t, sort, s_date, e_date, s_from, e_to)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []

# for i in urls:
#     url = articles_crawler(urls)
#     news_url.append(url)

while page <= maxpage_t :
    url_origin = "https://search.naver.com/search.naver?where=news&query=" + search + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
    url = articles_crawler(url_origin)
    news_url .append(url)
    page += 10

#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

100%|██████████| 6045/6045 [00:00<00:00, 1945711.59it/s]


In [23]:
# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
    news = requests.get(i, headers=headers, verify=False)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    # html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
    # news_date = html_date.attrs['data-date-time']
    # news_dates.append(news_date)

    try:
        html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

100%|██████████| 2035/2035 [15:39<00:00,  2.17it/s]


In [24]:
###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})
news_df

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.now() 
news_df.to_csv('//Users//yeijeong//big16//final_project//DATA//PAST_NEWS_DATA//{}_{}_date{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H%M%S'),s_from, e_to),encoding='utf-8-sig',index=False)

ValueError: All arrays must be of the same length